<a href="https://colab.research.google.com/github/Vbodake/Casting-Product-Defects-detection/blob/main/Casting_product_Defect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models as models
import numpy as np
import cv2
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"

class DefectClassifier(nn.Module):
    def __init__(self):
        super(DefectClassifier, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)

    def forward(self, x):
        return self.model(x)

@st.cache_resource
def load_model():
    model = DefectClassifier().to(device)
    model.load_state_dict(torch.load("/content/drive/MyDrive/best_model (2).pth", map_location=device))
    model.eval()
    return model

model = load_model()

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def resize_with_padding(image, target_size=(512, 512)):
    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_h, new_w = int(h * scale), int(w * scale)
    image_resized = cv2.resize(image, (new_w, new_h))
    pad_h, pad_w = target_size[0] - new_h, target_size[1] - new_w
    top, bottom = pad_h // 2, pad_h - pad_h // 2
    left, right = pad_w // 2, pad_w - pad_w // 2
    image_padded = cv2.copyMakeBorder(image_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    return image_padded

def predict(image_array):
    image = resize_with_padding(image_array)
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]
        predicted_class = np.argmax(probabilities)
    label_map = {0: "Non-Defective", 1: "Defective"}
    return label_map[predicted_class], probabilities[predicted_class]

st.title("🔍 Bullet Defect Classification")
st.write("Upload an image of a bullet to determine if it's **Defective** or **Non-Defective**.")

uploaded_file = st.file_uploader("Upload Bullet Image", type=["jpg", "jpeg", "png", "bmp"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    image_array = np.array(image)
    label, confidence = predict(image_array)
    st.image(image, caption=f"Prediction: {label} ({confidence:.2f})", use_container_width=True)


    if label == "Defective":
        st.markdown(f"<h3 style='color:red;'>⚠️ Defective ({confidence:.2f})</h3>", unsafe_allow_html=True)
    else:
        st.markdown(f"<h3 style='color:green;'>✅ Non-Defective ({confidence:.2f})</h3>", unsafe_allow_html=True)


Writing app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from pyngrok import conf
conf.get_default().auth_token = "2vl1VK4BcTRbSYmfXTU6fGvBT3i_5b4zPTSXc6UTrC6RqRGmi"



In [ ]:
from pyngrok import conf, ngrok
import subprocess
import time
import requests

# Set your ngrok auth token (Replace with your own token)
conf.get_default().auth_token = "2vl1VK4BcTRbSYmfXTU6fGvBT3i_5b4zPTSXc6UTrC6RqRGmi"
  # Replace with your ngrok auth token

# Kill any existing Streamlit process
!pkill -f streamlit

# Start Streamlit app (in background)
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait for Streamlit to be live
def wait_for_streamlit(timeout=60):
    for _ in range(timeout):
        try:
            r = requests.get("http://localhost:8501")
            if r.status_code == 200:
                return True
        except:
            pass
        time.sleep(1)
    return False

if wait_for_streamlit():
    # Open ngrok tunnel to Streamlit app
    public_url = ngrok.connect(8501)
    print(f"✅ Streamlit app is live at: {public_url}")
else:
    print("❌ Streamlit failed to launch.")


✅ Streamlit app is live at: NgrokTunnel: "https://813334801201.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
pip install pyngrok

In [ ]:
!streamlit run app.py


/bin/bash: line 1: streamlit: command not found


In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.9 MB/s eta 0:00:00
